In [1]:

import json

category_dict = {"안전/환경": 1,"미래": 2,"일자리": 3,"보건복지": 4,"정치개혁": 5,"경제민주화": 6,"인권/성평등": 7,"외교/통일/국방": 8,"육아/교육": 9,"문화/예술/체육/언론": 10,"반려동물": 11,"교통/건축/국토": 12,"행정": 13,"농산어촌": 14,"저출산/고령화대책": 15,"성장동력": 16,"기타": 17}
with open('/home/minwookje/다운로드/original.json') as f:
    data = json.load(f)

In [2]:
total = list()
for i in data:
    k = {"title":data[i]["title"],"category":data[i]["category"],"content": data[i]["content"]}
    total.append(k)
# 277030 문장    
print(len(total)+1)

277030


In [3]:
corpus = dict()
for k in category_dict.keys():
    sub = list()
    for i in total:
        if i["category"]== k:
#             print(type(i[k]))
#             print(i[k])
#             break
            sub.extend([i["content"]])
#             sub.extend([i["content"]+"  "])
#     break
    corpus[k] = sub
    


# CountVectorizer
``` python
# text2sparse_matrix(make term frequency matrix==TDM)
# 문서 집합에서 단어 토큰을 생성하고 각 단어의 수를 세어 BOW 인코딩한 벡터를 만든다.
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    #특정 단어를 포함한 문서의 비율. if min_df = 0.02(100개의 문서에서 2번등장한 단어까지 matrix에 포함)
    min_df=0,
    #max_df = 0.5, 51개 이상의 문서에서 등장한 단어들은 제외된다.
    max_df=1,
    #TDM에서 이용할 n-gram의 범위
    #default: uni, if (1,3)== uni,bi,trigram모두 이용
    ngram_range=(1,1),
    lowercase=True,
    #한국어에서는 이 tokenizer가 가장 중요하다. 
    tokenizer=lambda x:x.split())

corpus.iter_sent=False
# fit or fit_transform() input data type = list of str(DoublespaceLineCorpus output = str yield)
X = vectorizer.fit_transform(corpus)
print(X.shape) # (100, 7624)
```

# TfidfVectorizer
``` python
#TF-idf
# CountVectorizer를 이용하여 만들어진 TDM을 tfidf형식으로 변환.
# CountVectorizer와 비슷하지만 TF-IDF 방식으로 단어의 가중치를 조정한 BOW 벡터를 만든다
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    min_df=0,
    max_df=1,
    ngram_range=(1,1),
    lowercase=True,
    tokenizer=lambda x:x.split())

X = vectorizer.fit_transform(corpus)

```

# .fit() or .fit_transform()
```python
#  각 문서에 어떤 말들이 등장하였는지 정확히 살펴보기 위해 CountVectorizer 를 이용하여 
# term frequency matrix 를 먼저 만들어 저장을 해두고, 
# 필요시에 따라 tfidf 로 변환하여 이용.


vectorizer = CountVectorizer()
# sparse matrix 의 각 column 이 어떤 단어에 해당하는지에 대한 index 가 저장
# .fit()함수를 통하여 위의 결과 진행.
X = vectorizer.fit_transform(corpus)

transformer = TfidfTransformer()
X = transformer.fit_transform(X)

# .vocabulary_ {str:int}형식으로 각 단어 당 inx 저장 dict
vectorizer.vocabulary_
```
ex)
`{'19': 129,
 '1990': 149,
 '52': 478,
 '22': 265,
 '오패산터널': 5944,
 '총격전': 8150,
 ...
}`

> 각 idx가 어떤 단어인지를 저장하는 list or str생성 가능. 
띄어쓰기 기준으로 tokenizer 하여 비슷한 어절 모두 다른 단어로 학습됨.

```python
idx2vocab = [vocab for vocab, idx in sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1])]
print(idx2vocab[5537:5542])
# ['어려운', '어려움을', '어려움이', '어려워', '어려웠다']
```

> 이후 Vectorizer 저장  with pickling

```python
import pickle
with open('./vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)
```

> Vectorizer 에 의하여 만들어지는 term frequency matrix 는 scipy.sparse.csr.csr_matrix 형식, tdm 저장 법

```python

X = vectorizer.fit_transform(corpus)

#type
print(type(X))
# scipy.sparse.csr.csr_matrix


# save
from scipy.io import mmwrite

mtx_path = './x.mtx'
mmwrite(mtx_path, X)
```


> 저장된 matrix를 읽어올때, mmread()를 이용하면 coo matrix형식

> csr형식으로 변환하고 싶다면 tocsr()사용

```python
#load

from scipy.io import mmread

X = mmread(mtx_path)
print(type(X))
# scipy.sparse.coo.coo_matrix

X = X.tocsr()
print(type(X))
# scipy.sparse.csc.csc_matrix
```




# Scikit-learn vectorizer + KoNLPy

> konlpy의 tokenizer를 활용한 scikit-learn vectorizer를 사용하기 위해서 MyTokenizer를 생성해 준다.


In [5]:
from konlpy.tag import Okt as Twitter

class MyTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
#         pos = self.tagger.pos(sent)
        pos = self.tagger.nouns(sent)
#         pos = ['{}/{}'.format(word,tag) for word, tag in pos]
        pos = ['{}'.format(word) for word in pos]
#         여기에 noun추가
        return pos

# my_tokenizer = MyTokenizer(Twitter())

# sent = '이건테스트문장입니다.'
# print(my_tokenizer(sent))
# ['이건/Noun', '테스트/Noun', '문장/Noun', '입니/Adjective', '다/Eomi', './Punctuation']

> idx2vocab list를 생성해준다.

In [84]:
from konlpy.tag import Okt as Twitter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from scipy.io import mmwrite

# my_tokenizer = MyTokenizer(Twitter())
# if tf matrix
# vectorizer = CountVectorizer(tokenizer = my_tokenizer)
#  if tf-idf matrix
# vectorizer = TfidfVectorizer(tokenizer = my_tokenizer)
# x = vectorizer.fit_transform(corpus['안전/환경'])

# my_tokenizer = MyTokenizer(Twitter())
# corpus2 = dict()
# # # print(my_tokenizer(sent))
# for k in corpus.keys():
#     print(str(k)+" is tokenizing")
#     sub = list()
#     for i in corpus[k]:
#         tmp = my_tokenizer(i)
#     #     띄어쓰기로 분리된 
#         sub.append(' '.join(tmp))
#     corpus2[k] = sub
#     print(str(k)+" is tokenized")
print(len(corpus2))
for i in corpus2.keys():
    print(str(i)+" is vectorizing!")
    vectorizer = TfidfVectorizer()
#     vectorizer = TfidfVectorizer(
#         min_df=0,
#         max_df=1,
#         ngram_range=(1,1),
#         lowercase=True,
#         tokenizer=lambda x:x.split())
    pad_corpus = corpus2[i]
    pad_corpus.extend(corpus2['기타'])
    x = vectorizer.fit_transform(pad_corpus)
    print("vectorizing is finished")
    p_path = './padded_'+str(category_dict[i])+'.pkl'
#     p_path = './total.pkl'
    with open(p_path, 'wb') as f:
        pickle.dump(vectorizer, f)
#     mtx_path='./'+str(category_dict[i])+'.mtx'
#     mmwrite(mtx_path,x)
    print(p_path+" is saved")


# # idx2vocab = 단어들 리스트 생성
# idx2vocab = [vocab for vocab, idx in sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1])]
# # ['term/tag'] 
# # print(idx2vocab[4300:4400])
# print(idx2vocab[0])


17
안전/환경 is vectorizing!
vectorizing is finished
./padded_1.pkl is saved
미래 is vectorizing!
vectorizing is finished
./padded_2.pkl is saved
일자리 is vectorizing!
vectorizing is finished
./padded_3.pkl is saved
보건복지 is vectorizing!
vectorizing is finished
./padded_4.pkl is saved
정치개혁 is vectorizing!
vectorizing is finished
./padded_5.pkl is saved
경제민주화 is vectorizing!
vectorizing is finished
./padded_6.pkl is saved
인권/성평등 is vectorizing!
vectorizing is finished
./padded_7.pkl is saved
외교/통일/국방 is vectorizing!
vectorizing is finished
./padded_8.pkl is saved
육아/교육 is vectorizing!
vectorizing is finished
./padded_9.pkl is saved
문화/예술/체육/언론 is vectorizing!
vectorizing is finished
./padded_10.pkl is saved
반려동물 is vectorizing!
vectorizing is finished
./padded_11.pkl is saved
교통/건축/국토 is vectorizing!
vectorizing is finished
./padded_12.pkl is saved
행정 is vectorizing!
vectorizing is finished
./padded_13.pkl is saved
농산어촌 is vectorizing!
vectorizing is finished
./padded_14.pkl is saved
저출산/고령화대책 i

In [82]:
# my_tokenizer = MyTokenizer(Twitter())
# if tf matrix
# vectorizer = CountVectorizer(tokenizer = my_tokenizer)
#  if tf-idf matrix
# vectorizer = TfidfVectorizer(tokenizer = my_tokenizer)
# x = vectorizer.fit_transform(corpus['안전/환경'])

my_tokenizer = MyTokenizer(Twitter())

# # print(my_tokenizer(sent))

print(str(k)+" is tokenizing")
sub = list()
for k,i in enumerate(corpus['기타'][:10000]):
    tmp = my_tokenizer(i)
    #     띄어쓰기로 분리된 
    sub.append(' '.join(tmp))
    if k%1000 == 0:
        print(k)
corpus2['기타'] = sub
print(str(k)+" is tokenized")


1472 is tokenizing
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
9999 is tokenized


In [83]:
# corpus dict to (1,?) list 
# this one to make tfidf matrix and when test it'll be used
corpus_list = list()
for key,value in corpus2.items():
    corpus_list.extend(value)
print(str(i)+" is vectorizing!")
vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(corpus_list)
p_path = './total.pkl'
with open(p_path, 'wb') as f:
    pickle.dump(vectorizer, f)
#     mtx_path='./'+str(category_dict[i])+'.mtx'
#     mmwrite(mtx_path,x)
print(p_path+" is saved")
# #vectorizing 한 pickle 파일을 불러온다.
# with open('./1.pkl', 'rb') as f:
#     cate_1 = pickle.load(f)
# print(cate_1)
# print(len(corpus['미래']))
print(len(corpus_list))

 암호화폐 거래소 차단한다고 하는데 그럼 강원랜드 카지노 합법에 국민들에게 이로운지 알고 싶습니다 암호화폐 환경을 만들고 규제하시죠  강원랜드를 먼저 규제 하시죠  탁상행정을 하실려면 제대로 알고 접근하세요  is vectorizing!
./total.pkl is saved
252787


In [ ]:
# train: test = 8:2
t_rate = 0.8
train_len = round(len(X_train)*t_rate)

X_train = x[:train_len]
X_test = x[train_len:]
y_train = []

In [95]:
corpus2['기타'] = corpus2['기타'][:10000]

In [101]:
with open('corpus2.json','w') as f:
    json.dump(corpus2,f)

In [102]:
with open('corpus2.json','r') as f:
    json_corpus = json.load(f)

In [11]:
tmp= {'a':1,'b':78,'c':4}
sorted_by_value = sorted(tmp.items(), key=lambda kv: kv[1],reverse=True)  

In [12]:
print(dict(sorted_by_value))
# for k,v in sorted_by_value.items():
#     print(k)
#     print(v)
    
#     lk.extend(' '.join(i))
    
# with open(name,'w') as f:
#     json.dump(sorted_by_value,f)

{'b': 78, 'c': 4, 'a': 1}


In [8]:
k

['a', 1, 'c', 4, 'b', 78]